법안별 위원회 이름 count 추출 코드

In [3]:
import json
from collections import Counter
from pathlib import Path
import pandas as pd  # 요약표 출력을 위해 pandas 사용

# 1. 특정 폴더 경로 설정
folder_path = Path('/Users/seunghyunmoon/Code/Toy/vote_player_server/data/candidates')  # <-- 여기를 본인 폴더로 설정하세요

# 결과 기록용 리스트
summary_records = []
empty_bills_files = []  # collabills 없는 파일 목록

# 2. 폴더 내 모든 JSON 파일 순회
json_files = list(folder_path.glob('*.json'))

print(f"총 {len(json_files)}개의 JSON 파일을 처리합니다.\n")

for file_path in json_files:
    print(f"처리 중: {file_path.name}")

    # 2-1. 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 2-2. 기존 collabillsCommitteeStatistics가 있으면 삭제
    if 'collabillsCommitteeStatistics' in data:
        print(f"  - 기존 collabillsCommitteeStatistics 컬럼을 삭제합니다.")
        del data['collabillsCommitteeStatistics']

    # 2-3. collabills 데이터 수집
    collabills = data.get('collabills', [])
    bills_count = len(collabills)

    # 👉 bills가 아예 없는 경우 경고 출력 + 기록
    if bills_count == 0:
        print(f"  ⚠️  경고: 이 파일은 collabills 데이터가 없습니다!")
        empty_bills_files.append(file_path.name)

    counter = Counter()
    for collabill in collabills:
        nth = (collabill.get('nth') or '').replace('제', '').replace('대', '')
        committee = (collabill.get('committee') or '').strip()
        if nth and committee:
            counter[(nth, committee)] += 1

    # 2-4. Counter를 리스트로 변환
    collabills_statistics = [
        {
            'nth': f"{nth}대",
            'name': committee,
            'value': count
        }
        for (nth, committee), count in counter.items()
    ]

    statistics_count = len(collabills_statistics)

    # 2-5. 결과를 데이터에 추가
    data['collabillsCommitteeStatistics'] = collabills_statistics

    # 2-6. 결과 파일 저장 (원래 파일 덮어쓰기)
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"  - 저장 완료: {file_path.name}\n")

    # 2-7. 요약 기록 저장
    summary_records.append({
        '파일명': file_path.name,
        'collabills 항목 수': bills_count,
        '통계 항목 수': statistics_count
    })

# 3. 요약 테이블 출력
summary_df = pd.DataFrame(summary_records)

# 👉 '통계 항목 수' 기준 내림차순 정렬
summary_df = summary_df.sort_values(by='통계 항목 수', ascending=False)

print("✅ 처리 결과 요약:")
display(summary_df)

# 4. bills가 없는 파일 목록 출력
if empty_bills_files:
    print("\n⚠️ collabills 데이터가 없는 파일 목록:")
    for filename in empty_bills_files:
        print(f" - {filename}")
else:
    print("\n✅ 모든 파일에 collabills 데이터가 존재합니다!")


총 16개의 JSON 파일을 처리합니다.

처리 중: KangSeungKyoo.json
  - 저장 완료: KangSeungKyoo.json

처리 중: KANGYUJUNG.json
  - 저장 완료: KANGYUJUNG.json

처리 중: KANGKYUNGSOOK.json
  - 저장 완료: KANGKYUNGSOOK.json

처리 중: KWAKKYUTAEK.json
  - 저장 완료: KWAKKYUTAEK.json

처리 중: KANGSUNYOUNG.json
  - 저장 완료: KANGSUNYOUNG.json

처리 중: KANGMINKUK.json
  - 저장 완료: KANGMINKUK.json

처리 중: KANGSUNWOO.json
  - 저장 완료: KANGSUNWOO.json

처리 중: KANGJUNHYEON.json
  - 저장 완료: KANGJUNHYEON.json

처리 중: KANGDAESIK.json
  - 저장 완료: KANGDAESIK.json

처리 중: KANGHOONSIK.json
  - 저장 완료: KANGHOONSIK.json

처리 중: KUJAKEUN.json
  - 저장 완료: KUJAKEUN.json

처리 중: KANGMYOUNGGU.json
  - 저장 완료: KANGMYOUNGGU.json

처리 중: KWAKSANGEON.json
  - 저장 완료: KWAKSANGEON.json

처리 중: KOMINJUNG.json
  - 저장 완료: KOMINJUNG.json

처리 중: KOHDONGJIN.json
  - 저장 완료: KOHDONGJIN.json

처리 중: KANGDEUKGU.json
  - 저장 완료: KANGDEUKGU.json

✅ 처리 결과 요약:


,파일명,collabills 항목 수,통계 항목 수
9,KANGHOONSIK.json,2379,57
0,KangSeungKyoo.json,942,39
6,KANGSUNWOO.json,1406,38
10,KUJAKEUN.json,2573,38
7,KANGJUNHYEON.json,1758,37
8,KANGDAESIK.json,1378,37
13,KOMINJUNG.json,550,36
15,KANGDEUKGU.json,1608,35
5,KANGMINKUK.json,507,34
3,KWAKKYUTAEK.json,438,19



✅ 모든 파일에 collabills 데이터가 존재합니다!
